In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.SupportResistanceStrategy import SMAScanner
from src.classify import identify_pivots, calculate_sma_slope, calculate_cumulative_pivot_levels
from src.utils import get_last_market_day
import pandas_market_calendars as mcal
from src.constants import base_url

In [2]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

with open('cfg.txt') as f:
    token = f.read()

last_market_day = get_last_market_day()

# Step 1: Classify sentiment of the overall market and of the stock
- Bullish  
- Non-Bearish  
- Neutral  
- Non-Bullish  
- Bearish  

In [3]:
df_raw = yf.download(watchlist.split(' '), auto_adjust=False, interval='1d', period='6mo')

df = df_raw.stack([0,1], future_stack=True).reset_index().copy()
df.columns = ['Timestamp', 'Variable', 'Ticker', 'Value']
df = df.pivot(index=['Timestamp', 'Ticker'], columns='Variable', values='Value').reset_index().sort_values('Timestamp')
df.columns.name = None

[*********************100%***********************]  80 of 80 completed


In [4]:
result_dict = {}
for tick in watchlist.split(' '):
    df_tick = df[df['Ticker'] == tick].copy()
    sma_slope = calculate_sma_slope(df_tick, lookback=15) 
    df_pivots = identify_pivots(df_tick, threshold='atr', atr_scaling=0.8, use_high_low=True)
    pivot_trend = calculate_cumulative_pivot_levels(df_pivots, n_pivots=3) 
    result_dict[tick] = {'sma_slope':sma_slope.round(2), } | pivot_trend

In [5]:
output = pd.DataFrame(result_dict).T.dropna().sort_values('sma_slope')
output[output['higher_highs_higher_lows']]

,sma_slope,cumul_pivot_highs,cumul_pivot_lows,higher_highs_higher_lows,lower_highs_lower_lows,pivot_high_dates,pivot_low_dates
XLV,0.01,0.019713,0.028313,True,False,"['2025-05-20', '2025-06-13', '2025-07-01']","['2025-05-15', '2025-05-23', '2025-06-23']"
OIH,0.02,0.065376,0.083453,True,False,"['2025-05-12', '2025-06-17', '2025-07-11']","['2025-05-01', '2025-05-22', '2025-06-25']"
XLE,0.04,0.073533,0.052374,True,False,"['2025-04-28', '2025-05-13', '2025-06-18']","['2025-05-07', '2025-05-22', '2025-07-01']"
PLTR,0.04,0.109026,0.063867,True,False,"['2025-05-14', '2025-06-03', '2025-06-26']","['2025-05-23', '2025-06-09', '2025-07-01']"
GOOGL,0.05,0.025214,0.046636,True,False,"['2025-05-22', '2025-06-10', '2025-06-30']","['2025-06-03', '2025-06-23', '2025-07-09']"
SBUX,0.06,0.115507,0.093829,True,False,"['2025-05-28', '2025-06-11', '2025-07-09']","['2025-05-21', '2025-06-02', '2025-06-24']"
UPS,0.08,0.035058,0.057379,True,False,"['2025-05-12', '2025-06-11', '2025-07-03']","['2025-05-07', '2025-05-23', '2025-06-25']"
PYPL,0.08,0.06671,0.014935,True,False,"['2025-05-29', '2025-06-12', '2025-07-03']","['2025-06-03', '2025-06-18', '2025-07-11']"
RTX,0.08,0.13896,0.190099,True,False,"['2025-04-17', '2025-05-21', '2025-06-18']","['2025-04-22', '2025-05-28', '2025-06-25']"
XOM,0.09,0.069346,0.036769,True,False,"['2025-04-23', '2025-05-13', '2025-06-23']","['2025-05-06', '2025-05-30', '2025-07-01']"


In [ ]:
pivot_high_periods = df_pivots[df_pivots["pivot_type"] == "high"]

str([x.strftime('%Y-%m-%d') for x in pivot_high_periods['Timestamp'].iloc[-3:].tolist()])


# Step 2: Characterize recent price action  
- Where is support/resistance?  
- Is the stock extended?  
- Higher pivot highs/higher pivot lows?  
- Lower pivot highs/lower pivot lows?  
- Chopping in a range?  
- What has a better chance of happening?  

# Step 3: Volatility Analysis

In [ ]:
import requests
def get_ivrank(ticker:str, token:str) -> dict:
    endpoint = "ivrank"
    url = f"{base_url}{endpoint}"
    querystring = {"token":token,"ticker":ticker}
    response = requests.request("GET", url, params=querystring)
    return pd.DataFrame(response.json()['data'])



In [ ]:
endpoint = "cores"
url = f"{base_url}{endpoint}"
querystring = {"token":token,"ticker":'AAPL'}
response = requests.request("GET", url, params=querystring)

In [ ]:
response.json()['data'][0]['ivHvXernRatio']

In [ ]:
ivr_data = get_ivrank(watchlist.replace(' ',','), token)

In [ ]:
output = pd.DataFrame(result_dict).T.sort_values('slope')
output = output.merge(ivr_data[['ticker','iv', 'ivRank1m', 'ivPct1m','ivRank1y','ivPct1y']], left_index=True, right_on='ticker', )
output.index = output['ticker']
output = output.drop('ticker', axis=1).sort_values('ivRank1y')
pd.set_option('display.max_rows', 200)
output

# Archive

In [ ]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, period='2y')

In [ ]:
scanner = SMAScanner(sma_periods=[50, 200], return_window=20)
output = scanner.run_scan(df_raw) # type: ignore
output

In [ ]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, interval='60m', period='3mo')

In [ ]:
df = df_raw.stack(level=[0,1], future_stack=True).reset_index()
df.columns=['Timestamp', 'Variable', 'Ticker', 'Value']
df

In [ ]:
# assert set(df.columns) >= {'Timestamp', 'Variable', 'Ticker', 'Value'}
df_wide = df.pivot(index=['Timestamp', 'Ticker'], columns='Variable', values='Value').reset_index()
results = {}

window = 33
buffer = 0.01
for ticker, df_ticker in df_wide.groupby('Ticker'):
    df_ticker = df_ticker.sort_values('Timestamp').copy()
    df_ticker.set_index("Timestamp", inplace=True)
    if 'Open' not in df_ticker or 'Close' not in df_ticker:
        continue
    df_ticker = df_ticker.dropna(subset=['Open', 'Close'])
    body_top = df_ticker[['Open', 'Close']].max(axis=1)
    body_bottom = df_ticker[['Open', 'Close']].min(axis=1)
    pivot_high = (body_top == body_top.rolling(window, center=True).max())
    pivot_low = (body_bottom == body_bottom.rolling(window, center=True).min())
    df_ticker['pivot_high'] = pivot_high
    df_ticker['pivot_low'] = pivot_low
    resistance = df_ticker.loc[df_ticker['pivot_high'], ['Open', 'Close']].max(axis=1).tolist()
    support = df_ticker.loc[df_ticker['pivot_low'], ['Open', 'Close']].min(axis=1).tolist()

    # Group close levels
    def group_levels(levels):
        levels = sorted(levels)
        grouped = []
        for lvl in levels:
            if not grouped or abs(lvl - grouped[-1]) / grouped[-1] > buffer:
                grouped.append(lvl)
        return grouped

    results[ticker] = {
        "support": group_levels(support),
        "resistance": group_levels(resistance)
    }


In [ ]:
df_wide.groupby('Ticker').resample('1d', on='Timestamp')['Close'].last().dropna()